In [1]:
! pip install tensorflow-addons

In [2]:
! pip install tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 11.4 MB/s eta 0:00:00


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow_addons as tfa
from math import pi, ceil

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
ndim=20

In [5]:
class AEBase(keras.Model):
    def __init__(self, **kwargs):
        super(AEBase, self).__init__(**kwargs)
        self.encoder = self.construct_encoder()
        self.decoder = self.construct_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z, flat_encoded = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=(0, 1)
                )
            )
        grads = tape.gradient(reconstruction_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(reconstruction_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
        }

In [6]:
class CNNAE(AEBase):
    def __init__(self, latent_dim=12, encoder_hiddens=[256, 128, 64], decoder_hiddens=[64, 128, 256], kernel_size=2, stride=1, series_len=None):
        self.latent_dim = latent_dim
        self.encoder_hiddens = encoder_hiddens
        self.decoder_hiddens = decoder_hiddens
        self.series_len = series_len
        self.kernel_size = kernel_size
        self.stride = stride
        super(CNNAE, self).__init__()

    def construct_encoder(self):
        encoder_input = keras.Input((None, self.series_len))
        #x = keras.layers.Masking(mask_value=0.0)(encoder_input)
        x = encoder_input
        for filters in self.encoder_hiddens:
            x = layers.Conv1D(filters=filters, kernel_size=self.kernel_size, padding="same", strides=self.stride, activation="linear")(x)

        encoded = layers.Conv1D(filters=self.latent_dim, kernel_size=self.kernel_size, padding="same", strides=self.stride, activation="linear", name="encoded")(x)
        flat_encoded = Flatten()(encoded)

        encoder = keras.Model(encoder_input, [encoded, flat_encoded], name="encoder")
        return encoder

    def construct_decoder(self):
        decoder_input = keras.Input((None, self.latent_dim))
        x = decoder_input

        for filters in self.decoder_hiddens:
            x = layers.Conv1DTranspose(filters=filters, kernel_size=self.kernel_size, padding="same", strides=self.stride, activation="linear")(x)

        decoded = layers.Conv1DTranspose(filters=1, kernel_size=self.kernel_size, padding="same")(x)

        decoder = keras.Model(decoder_input, decoded, name="decoder")
        return decoder

In [7]:
nn5 = pd.read_csv("/kaggle/input/nn5-dataset/nn51.csv", header=None)
nn5.head()

,0,1,2,3,4,5,6,7,8,9,...,781,782,783,784,785,786,787,788,789,790
0,13.407029,14.725057,20.564059,34.708050,26.629819,16.609977,15.320295,11.607143,19.883787,23.767007,...,50.141723,33.701814,28.656463,26.417234,27.253401,44.373583,65.206916,49.744898,34.481293,32.667234
1,11.550454,13.591270,15.036848,21.570295,19.444444,0.000000,9.722222,12.244898,15.504535,18.934240,...,34.665533,13.988095,13.874717,16.326531,17.488662,20.663265,27.097506,32.171202,13.676304,16.369048
2,5.640590,14.399093,24.418934,28.784014,20.620748,13.803855,11.536281,10.742630,14.824263,25.212585,...,38.364512,17.928005,18.041383,16.978458,22.293084,36.522109,42.786281,39.271542,17.446145,17.148526
3,13.180272,8.446712,19.515306,28.883220,19.472789,0.000000,7.355442,10.827664,15.617914,21.159297,...,31.420068,17.276077,15.674603,12.471655,17.928005,25.028345,45.308957,32.242063,15.943878,16.638322
4,9.778912,10.813492,21.612812,38.520408,24.744898,12.329932,12.996032,11.040249,7.950680,19.515306,...,33.758503,18.990930,16.865079,16.000567,15.079365,20.833333,40.646259,33.304989,16.666667,14.101474


In [8]:
nn5

,0,1,2,3,4,5,6,7,8,9,...,781,782,783,784,785,786,787,788,789,790
0,13.407029,14.725057,20.564059,34.708050,26.629819,16.609977,15.320295,11.607143,19.883787,23.767007,...,50.141723,33.701814,28.656463,26.417234,27.253401,44.373583,65.206916,49.744898,34.481293,32.667234
1,11.550454,13.591270,15.036848,21.570295,19.444444,0.000000,9.722222,12.244898,15.504535,18.934240,...,34.665533,13.988095,13.874717,16.326531,17.488662,20.663265,27.097506,32.171202,13.676304,16.369048
2,5.640590,14.399093,24.418934,28.784014,20.620748,13.803855,11.536281,10.742630,14.824263,25.212585,...,38.364512,17.928005,18.041383,16.978458,22.293084,36.522109,42.786281,39.271542,17.446145,17.148526
3,13.180272,8.446712,19.515306,28.883220,19.472789,0.000000,7.355442,10.827664,15.617914,21.159297,...,31.420068,17.276077,15.674603,12.471655,17.928005,25.028345,45.308957,32.242063,15.943878,16.638322
4,9.778912,10.813492,21.612812,38.520408,24.744898,12.329932,12.996032,11.040249,7.950680,19.515306,...,33.758503,18.990930,16.865079,16.000567,15.079365,20.833333,40.646259,33.304989,16.666667,14.101474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,8.857710,11.380385,11.777211,18.820862,24.305556,6.760204,11.493764,10.572562,11.210317,15.830499,...,26.431406,2.295918,13.605442,13.477891,12.372449,0.070862,20.918367,26.643991,4.733560,12.131519
107,9.297052,12.429138,13.251134,24.815760,19.231859,9.240363,11.096939,10.926871,11.295351,17.800454,...,21.981293,11.252834,11.947279,9.991497,12.740930,12.996032,23.993764,21.485261,10.076531,13.959751
108,10.416667,14.923469,25.864512,37.996032,33.290816,17.261905,13.180272,15.235261,16.128118,23.497732,...,38.860544,20.875850,20.464853,19.175170,20.620748,36.507937,51.785714,42.488662,18.409864,22.307256
109,11.635488,10.586735,12.570862,17.517007,24.291383,14.852608,16.638322,13.180272,12.329932,14.328231,...,40.632086,22.973356,24.489796,17.403628,23.242630,29.209184,31.164966,40.816327,25.751134,26.984127


In [9]:
from datetime import datetime, timedelta
start_date = datetime(1996, 3, 18)

date_range = [start_date + timedelta(days=i) for i in range(nn5.shape[1])]

# Convert the date range to days of the week
days_of_week = [date.strftime('%A') for date in date_range]
nn5.columns = days_of_week

In [10]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with columns as days of the week

# Function to replace zeros with the median of non-zero values for each day of the week
def replace_zeros_with_median(series):
    # Iterate over unique days of the week
    for day in nn5.columns:
        # Extract non-zero values for the specific day
        non_zero_values = series[series.index == day].replace(0, np.nan).dropna()
        
        # Calculate the median of non-zero values
        median_value = non_zero_values.median()
        
        # Replace zeros with the median for the specific day
        series.loc[series.index == day] = series.loc[series.index == day].replace(0, median_value)

# Apply the function to each row of the DataFrame
nn5.apply(replace_zeros_with_median, axis=1)

# Now, df has zeros replaced with the median of non-zero values for each day of the week in each series


0      None
1      None
2      None
3      None
4      None
       ... 
106    None
107    None
108    None
109    None
110    None
Length: 111, dtype: object

In [11]:
nn5

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,...,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,13.407029,14.725057,20.564059,34.708050,26.629819,16.609977,15.320295,11.607143,19.883787,23.767007,...,50.141723,33.701814,28.656463,26.417234,27.253401,44.373583,65.206916,49.744898,34.481293,32.667234
1,11.550454,13.591270,15.036848,21.570295,19.444444,13.591270,9.722222,12.244898,15.504535,18.934240,...,34.665533,13.988095,13.874717,16.326531,17.488662,20.663265,27.097506,32.171202,13.676304,16.369048
2,5.640590,14.399093,24.418934,28.784014,20.620748,13.803855,11.536281,10.742630,14.824263,25.212585,...,38.364512,17.928005,18.041383,16.978458,22.293084,36.522109,42.786281,39.271542,17.446145,17.148526
3,13.180272,8.446712,19.515306,28.883220,19.472789,15.306122,7.355442,10.827664,15.617914,21.159297,...,31.420068,17.276077,15.674603,12.471655,17.928005,25.028345,45.308957,32.242063,15.943878,16.638322
4,9.778912,10.813492,21.612812,38.520408,24.744898,12.329932,12.996032,11.040249,7.950680,19.515306,...,33.758503,18.990930,16.865079,16.000567,15.079365,20.833333,40.646259,33.304989,16.666667,14.101474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,8.857710,11.380385,11.777211,18.820862,24.305556,6.760204,11.493764,10.572562,11.210317,15.830499,...,26.431406,2.295918,13.605442,13.477891,12.372449,0.070862,20.918367,26.643991,4.733560,12.131519
107,9.297052,12.429138,13.251134,24.815760,19.231859,9.240363,11.096939,10.926871,11.295351,17.800454,...,21.981293,11.252834,11.947279,9.991497,12.740930,12.996032,23.993764,21.485261,10.076531,13.959751
108,10.416667,14.923469,25.864512,37.996032,33.290816,17.261905,13.180272,15.235261,16.128118,23.497732,...,38.860544,20.875850,20.464853,19.175170,20.620748,36.507937,51.785714,42.488662,18.409864,22.307256
109,11.635488,10.586735,12.570862,17.517007,24.291383,14.852608,16.638322,13.180272,12.329932,14.328231,...,40.632086,22.973356,24.489796,17.403628,23.242630,29.209184,31.164966,40.816327,25.751134,26.984127


In [12]:
def preprocessing(data):
    ts_train = []

    for i in range(data.shape[0]):
        # for normalization of each series
        # mean_temp=np.mean(np.array(list(data.iloc[i][6:].dropna())))
        # temp = np.array(list(data.iloc[i][6:].dropna())/mean_temp)
        # temp = np.array(list(data.iloc[i][6:].dropna()))
        # temp=np.log(temp + 1)
        temp=np.array(list(data.iloc[i].dropna()))
        temp=temp.reshape(1,len(temp),1)
        temp2 = TimeSeriesScalerMeanVariance().fit_transform(temp)
        # temp = TimeSeriesScalerMeanVariance().fit_transform(np.array(list(m3.iloc[0][6:].dropna())).reshape(-1,1))

        ts_train.append(temp2.reshape(-1,1))

    return ts_train

In [13]:
dataset_nn5 = preprocessing(nn5)
dataset_nn5 = np.array(dataset_nn5)

In [14]:
max_seq_length_nn5 = max(len(seq) for seq in dataset_nn5)

In [15]:
max(len(seq) for seq in dataset_nn5)

791

In [16]:
reshaped_array_nn5 = dataset_nn5.reshape(dataset_nn5.shape[0], 1, dataset_nn5.shape[1])

In [17]:
print(reshaped_array_nn5.shape)

(111, 1, 791)


In [18]:
dim = [8, 16, 32]

In [19]:
for item in dim:
    print("****************************************************")
    print("Checking the dimension: ", item)
    print("****************************************************")
    ae_nn5=CNNAE(latent_dim=item, series_len=max_seq_length_nn5)
    ae_nn5.compile(optimizer=keras.optimizers.Adam(),loss="mse")
    ae_nn5.fit(reshaped_array_nn5, epochs=100, batch_size=10)
    _, Features_nn5 = ae_nn5.encoder(reshaped_array_nn5)
    path = "Features_nn5_CNN_"+str(item)+".csv"
    np.savetxt(path, Features_nn5, delimiter=",")

****************************************************
Checking the dimension:  8
****************************************************
Epoch 1/100
12/12 [==============================] - 7s 88ms/step - loss: 10.7969 - reconstruction_loss: 10.0335
Epoch 2/100
12/12 [==============================] - 1s 75ms/step - loss: 10.0882 - reconstruction_loss: 9.3975
Epoch 3/100
12/12 [==============================] - 1s 77ms/step - loss: 9.9152 - reconstruction_loss: 9.2763
Epoch 4/100
12/12 [==============================] - 1s 77ms/step - loss: 9.9010 - reconstruction_loss: 9.2616
Epoch 5/100
12/12 [==============================] - 1s 79ms/step - loss: 9.8909 - reconstruction_loss: 9.2548
Epoch 6/100
12/12 [==============================] - 1s 77ms/step - loss: 9.8872 - reconstruction_loss: 9.2521
Epoch 7/100
12/12 [==============================] - 1s 77ms/step - loss: 9.8858 - reconstruction_loss: 9.2510
Epoch 8/100
12/12 [==============================] - 1s 81ms/step - loss: 9.8852 - reco